In [6]:
import os
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers
from keras.layers import Dense, LSTM, Embedding, LeakyReLU, BatchNormalization, Dropout, Input, ReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop 
import optuna

### Definitions

In [7]:
# Function to scale data between 0 and 1
def scale_data(data):
    min_val = data.min().min()  # Get the global minimum
    max_val = data.max().max()  # Get the global maximum
    scaled_data = (data - min_val) / (max_val - min_val)  # Apply min-max scaling
    return scaled_data

def build_generator(latent_dim, data_shape, num_layers=4, num_neurons=256, activation='relu', dropout=0.2):
    model = Sequential()
    
    model.add(Dense(num_neurons, input_dim=latent_dim))
    model.add(BatchNormalization())
    model.add(ReLU())

    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons))
        model.add(BatchNormalization())
        model.add(ReLU())

    model.add(Dense(data_shape, activation='sigmoid'))
    
    noise = Input(shape=(latent_dim,))
    generated_data = model(noise)
    
    return Model(noise, generated_data)

def build_discriminator(data_shape, num_layers=4, num_neurons=256, activation='relu', dropout=0.2):
    model = Sequential()
    
    model.add(Dense(num_neurons, input_dim=data_shape))
    model.add(LeakyReLU())

    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons))
        model.add(LeakyReLU())  

    model.add(Dense(1, activation='sigmoid'))
    
    data = Input(shape=(data_shape,))
    validity = model(data)
    
    return Model(data, validity)

def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

def gradient_penalty(real_data, fake_data, discriminator):
    alpha = tf.random.normal([real_data.shape[0], 1], 0.0, 1.0)
    interpolated = alpha * real_data + (1 - alpha) * fake_data
    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        prediction = discriminator(interpolated)
    gradients = tape.gradient(prediction, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1]))
    penalty = tf.reduce_mean((norm - 1.0) ** 2)
    return penalty

def compute_gradient_norm(model, real_data, fake_data):
    with tf.GradientTape() as tape:
        # Obliczanie straty dyskryminatora
        real_output = model(real_data, training=True)
        fake_output = model(fake_data, training=True)
        loss = wasserstein_loss(real_output, fake_output)

    # Obliczanie gradientów względem wag dyskryminatora
    gradients = tape.gradient(loss, model.trainable_variables)
    # Obliczanie normy gradientów
    gradient_norm = tf.linalg.global_norm(gradients)
    return gradient_norm.numpy()

def inverse_scale_data(generated_data, original_min, original_max):
    # Przeskaluj dane z powrotem do oryginalnego zakresu
    inversed_data = generated_data * (original_max - original_min) + original_min
    return inversed_data

def generate_data(generator, n_samples, latent_dim, original_min, original_max):
    noise = np.random.normal(0, 1, size=(n_samples, latent_dim))
    generated_data = generator.predict(noise)
    generated_data = inverse_scale_data(generated_data, original_min, original_max)
    return generated_data

# FID score function
def calculate_fid(real_data, generated_data):
    # Przygotowanie danych
    real_data = real_data.astype('float32')
    generated_data = generated_data.astype('float32')
    # Obliczanie średniej i macierzy kowariancji dla prawdziwych danych
    mu1, sigma1 = real_data.mean(axis=0), np.cov(real_data, rowvar=False)
    # Obliczanie średniej i macierzy kowariancji dla wygenerowanych danych
    mu2, sigma2 = generated_data.mean(axis=0), np.cov(generated_data, rowvar=False)
    # Obliczanie sumy kwadratów różnicy średnich
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # Obliczanie pierwiastka z iloczynu macierzy kowariancji
    covmean = sqrtm(sigma1.dot(sigma2))
    # Sprawdzanie, czy macierz kowariancji jest poprawna
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # Obliczanie FID score
    fid_score = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid_score

def train(generator, discriminator, combined, data, latent_dim, epochs, batch_size=128, n_critic=5, lambda_gp=10.0, save_interval=50):
    # Ładowanie i skalowanie danych
    X_train = data.values

    # Initialize lists to store loss values
    d_loss_list = []
    g_loss_list = []
    gradient_norm_list = []
    gradient_norm = 0
    # Pętla po epokach
    for epoch in range(epochs):
        start = time.time()
        for _ in range(n_critic):
            # ---------------------
            #  Trenowanie dyskryminatora
            # ---------------------
            
            discriminator.trainable = True

            # Wybieranie losowych próbek
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_data = X_train[idx]

            # Generowanie nowego szumu
            noise = np.random.normal(0, 1, (batch_size, latent_dim))

            # Generowanie nowych danych
            generated_data = generator.predict(noise)

            # Trenowanie dyskryminatora
            d_loss_real = discriminator.train_on_batch(real_data, -np.ones((batch_size, 1), dtype=np.float32))
            d_loss_fake = discriminator.train_on_batch(generated_data, np.ones((batch_size, 1), dtype=np.float32))
            d_loss = d_loss_fake - d_loss_real + lambda_gp * gradient_penalty(real_data, generated_data, discriminator)

            gradient_norm = compute_gradient_norm(discriminator, real_data, generated_data)
            gradient_norm_list.append(gradient_norm)

        # ---------------------
        #  Trenowanie generatora
        # ---------------------

        discriminator.trainable = False
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Chcemy, aby dyskryminator uznał wygenerowane dane za prawdziwe
        valid_y = np.array([1] * batch_size, dtype=np.float32)

        # Trenowanie generatora
        g_loss = combined.train_on_batch(noise, valid_y)

        # Append loss values to lists
        d_loss_list.append(d_loss)
        g_loss_list.append(g_loss)

        # Zapisywanie postępów
        if epoch % save_interval == 0:
            print("%d [D loss: %f] [G loss: %f] [D gradient norm: %f]" % (epoch, d_loss, g_loss, gradient_norm))
            print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
            #checkpoint.save(file_prefix = checkpoint_prefix)
    
    return d_loss_list, g_loss_list, gradient_norm_list


### Preparation

In [15]:
# Load the data
data_path = 'original_data/GSE158508_normalized_counts.tsv'
data = pd.read_csv(data_path, sep='\t', index_col=0)

data_shape = data.shape[1]  # 69 columns

# Save columns names for later use
col_names = data.columns.values

original_min = data.min().min()
original_max = data.max().max()

original_data = data
data = scale_data(data)

### Training

In [16]:
def objective(trial):
    # Sugerowanie hiperparametrów
    num_layers_generator = trial.suggest_int('num_layers_generator', 2, 6)
    num_layers_discriminator = trial.suggest_int('num_layers_discriminator', 2, 6)
    neurons_per_layer_generator = trial.suggest_categorical('neurons_per_layer_generator', [64, 128, 256, 512])
    neurons_per_layer_discriminator = trial.suggest_categorical('neurons_per_layer_discriminator', [64, 128, 256, 512])
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3, log=True)
    n_critic = trial.suggest_int('n_critic', 1, 5)
    lambda_gp = trial.suggest_float('lambda_gp', 1e-2, 1e2, log=True)
    latent_dim = trial.suggest_int('latent_dim', 32, 128)
    batch_size = 32
    optimizer = trial.suggest_categorical('optimizer', ['RMSprop', 'Adam'])
    if optimizer == 'RMSprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)
    
    discriminator = build_discriminator(data_shape, num_layers=num_layers_discriminator, num_neurons=neurons_per_layer_discriminator)
    discriminator.compile(loss=wasserstein_loss, optimizer=opt)
    generator = build_generator(latent_dim, data_shape, num_layers=num_layers_generator, num_neurons=neurons_per_layer_generator)
    z = Input(shape=(latent_dim,))
    generated_data = generator(z)
    discriminator.trainable = False
    validity = discriminator(generated_data)
    combined = Model(z, validity)
    combined.compile(loss=wasserstein_loss, optimizer=opt)

    # Trenowanie modelu
    train(generator, discriminator, combined, data, latent_dim, epochs=5000, batch_size=batch_size, n_critic=n_critic, lambda_gp=lambda_gp, save_interval=100)

    synthetic_data = generate_data(generator, n_samples=57736, latent_dim=latent_dim, original_min=original_min, original_max=original_max)
    df = pd.DataFrame(synthetic_data)
    df.columns = col_names
    df.to_csv('synthetic_data/generated_data.tsv', sep='\t', index=False, header=True)
    df = pd.read_csv('synthetic_data/generated_data.tsv', sep='\t')
    synthetic_data = df

    # Obliczanie metryki, która ma być optymalizowana
    fid_score = calculate_fid(original_data, synthetic_data)

    return fid_score

In [17]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print(' Value: ', trial.value)
print(' Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2023-11-11 23:34:18,327] A new study created in memory with name: no-name-3335bdbf-c18b-4af6-8615-24f448070d07


0 [D loss: 41.106987] [G loss: 0.484376] [D gradient norm: 0.575881]
Time for epoch 1 is 1.774334192276001 sec
100 [D loss: 41.204723] [G loss: 0.005452] [D gradient norm: 0.061868]
Time for epoch 101 is 0.37404513359069824 sec
200 [D loss: 48.340004] [G loss: 0.001774] [D gradient norm: 0.184417]
Time for epoch 201 is 0.4197566509246826 sec
300 [D loss: 83.167107] [G loss: 0.000307] [D gradient norm: 0.031936]
Time for epoch 301 is 0.3801412582397461 sec
400 [D loss: 67.965309] [G loss: 0.000411] [D gradient norm: 0.040873]
Time for epoch 401 is 0.32248687744140625 sec
500 [D loss: 60.609570] [G loss: 0.000042] [D gradient norm: 0.004021]
Time for epoch 501 is 0.33448171615600586 sec
600 [D loss: 64.912117] [G loss: 0.000035] [D gradient norm: 0.008058]
Time for epoch 601 is 0.6025094985961914 sec
700 [D loss: 61.974445] [G loss: 0.000024] [D gradient norm: 0.001747]
Time for epoch 701 is 0.47564244270324707 sec
800 [D loss: 70.976601] [G loss: 0.000017] [D gradient norm: 0.000823]
Ti

[I 2023-11-12 00:12:24,015] Trial 0 finished with value: 5630.636550454829 and parameters: {'num_layers_generator': 5, 'num_layers_discriminator': 4, 'neurons_per_layer_generator': 256, 'neurons_per_layer_discriminator': 128, 'learning_rate': 2.857417833645217e-05, 'n_critic': 3, 'lambda_gp': 47.36625865879427, 'latent_dim': 57, 'optimizer': 'Adam'}. Best is trial 0 with value: 5630.636550454829.


0 [D loss: 1.649162] [G loss: 0.490740] [D gradient norm: 0.528372]
Time for epoch 1 is 3.362375259399414 sec
100 [D loss: 1.175102] [G loss: 0.008779] [D gradient norm: 0.062632]
Time for epoch 101 is 0.7754716873168945 sec
200 [D loss: 1.413219] [G loss: 0.003679] [D gradient norm: 0.099189]
Time for epoch 201 is 0.7162075042724609 sec
300 [D loss: 1.622063] [G loss: 0.001381] [D gradient norm: 0.229910]
Time for epoch 301 is 0.7513277530670166 sec
400 [D loss: 2.300693] [G loss: 0.000444] [D gradient norm: 0.053428]
Time for epoch 401 is 0.715470552444458 sec
500 [D loss: 2.763347] [G loss: 0.000009] [D gradient norm: 0.003511]
Time for epoch 501 is 0.706599235534668 sec
600 [D loss: 2.229146] [G loss: 0.000001] [D gradient norm: 0.002277]
Time for epoch 601 is 0.744534969329834 sec
700 [D loss: 3.323611] [G loss: 0.000000] [D gradient norm: 0.000252]
Time for epoch 701 is 0.7530486583709717 sec
800 [D loss: 1.793557] [G loss: 0.000000] [D gradient norm: 0.000012]
Time for epoch 801

[I 2023-11-12 01:14:04,596] Trial 1 finished with value: 5424.101562877995 and parameters: {'num_layers_generator': 4, 'num_layers_discriminator': 5, 'neurons_per_layer_generator': 256, 'neurons_per_layer_discriminator': 128, 'learning_rate': 1.3220810900803522e-05, 'n_critic': 4, 'lambda_gp': 0.7537767976931569, 'latent_dim': 125, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 5424.101562877995.


...

0 [D loss: 6.476118] [G loss: 0.456582] [D gradient norm: 0.582217]
Time for epoch 1 is 4.892173767089844 sec
100 [D loss: 6.145654] [G loss: 0.326115] [D gradient norm: 0.564590]
Time for epoch 101 is 0.8150250911712646 sec
200 [D loss: 5.992517] [G loss: 0.184608] [D gradient norm: 0.497817]
Time for epoch 201 is 0.7939791679382324 sec
300 [D loss: 6.044525] [G loss: 0.073966] [D gradient norm: 0.310507]
Time for epoch 301 is 0.792595624923706 sec
400 [D loss: 5.671844] [G loss: 0.022487] [D gradient norm: 0.119827]
Time for epoch 401 is 0.8075876235961914 sec
500 [D loss: 5.912882] [G loss: 0.008767] [D gradient norm: 0.057198]
Time for epoch 501 is 0.8475444316864014 sec
600 [D loss: 5.529698] [G loss: 0.005725] [D gradient norm: 0.048124]
Time for epoch 601 is 0.8134689331054688 sec
700 [D loss: 5.755090] [G loss: 0.004682] [D gradient norm: 0.035590]
Time for epoch 701 is 0.8325965404510498 sec
800 [D loss: 5.831905] [G loss: 0.004220] [D gradient norm: 0.040941]
Time for epoch 8

[I 2023-11-12 11:16:16,246] Trial 11 finished with value: 4016.5606941889614 and parameters: {'num_layers_generator': 6, 'num_layers_discriminator': 6, 'neurons_per_layer_generator': 64, 'neurons_per_layer_discriminator': 256, 'learning_rate': 1.074351501589858e-06, 'n_critic': 4, 'lambda_gp': 6.001204144304466, 'latent_dim': 35, 'optimizer': 'RMSprop'}. Best is trial 11 with value: 4016.5606941889614.


0 [D loss: 4.844946] [G loss: 0.493808] [D gradient norm: 0.757572]
Time for epoch 1 is 4.347909450531006 sec
100 [D loss: 4.731856] [G loss: 0.355537] [D gradient norm: 0.674791]
Time for epoch 101 is 0.7118070125579834 sec
200 [D loss: 4.718980] [G loss: 0.229759] [D gradient norm: 0.533294]
Time for epoch 201 is 0.6606955528259277 sec
300 [D loss: 4.672799] [G loss: 0.134445] [D gradient norm: 0.391235]
Time for epoch 301 is 1.3435606956481934 sec
400 [D loss: 4.427530] [G loss: 0.069159] [D gradient norm: 0.242875]
Time for epoch 401 is 0.6990177631378174 sec
500 [D loss: 4.751837] [G loss: 0.034453] [D gradient norm: 0.139026]
Time for epoch 501 is 0.6817755699157715 sec
600 [D loss: 4.562175] [G loss: 0.017685] [D gradient norm: 0.081571]
Time for epoch 601 is 0.6764528751373291 sec
700 [D loss: 4.573939] [G loss: 0.010533] [D gradient norm: 0.053210]
Time for epoch 701 is 0.6808159351348877 sec
800 [D loss: 4.626626] [G loss: 0.007400] [D gradient norm: 0.040631]
Time for epoch 

[W 2023-11-12 11:37:15,383] Trial 12 failed with parameters: {'num_layers_generator': 6, 'num_layers_discriminator': 3, 'neurons_per_layer_generator': 64, 'neurons_per_layer_discriminator': 256, 'learning_rate': 1.1521705846514664e-06, 'n_critic': 4, 'lambda_gp': 4.6143783643002045, 'latent_dim': 48, 'optimizer': 'RMSprop'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\Apps\anaconda3\envs\krzysiu\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\KrzysztofJedraszek\AppData\Local\Temp\ipykernel_15052\2012784466.py", line 29, in objective
    train(generator, discriminator, combined, data, latent_dim, epochs=5000, batch_size=batch_size, n_critic=n_critic, lambda_gp=lambda_gp, save_interval=100)
  File "C:\Users\KrzysztofJedraszek\AppData\Local\Temp\ipykernel_15052\24153529.py", line 132, in train
    d_loss_real = discriminator.train_on_batch(real_data, -np.ones((bat

KeyboardInterrupt: 